# ***Complete Guide to Generative AI for Data Analysis and Data Science - Capstone Project***

---


**Capstone Project** <br>
*   Assume you are an analyst within a company manufacturing electric vehicles.
*   The company has multiple suppliers that provide multiple parts.
*   Executives have asked you to analyze inventory management.
*   Can the company reduce costs by using a single supplier instead of three suppliers?

*Project Tasks:* <br>

1.   Describe the structure of the order data provided by the inventory management team.
2.   Clean the dataset to remove duplicate rows or rows with missing data.
3.   Compute the average time to delivery by supplier (delivery time is the difference between when order date and expected delivery date)
4.   Create an inventory management simulation. Randomly select demand from normal distribution (mean: 50, std: 10). Assume the holding cost is 10 dollars per unit per day. Assume the shortage cost is 50 dollars per unit per day. For each supplier, use their average days to delivery as the order lead time. For each supplier, run a 180-day simulation and calculate the inventory management costs for that supplier.
5.   Determine which supplier has the lowest cost.

*1. Describe the structure of the order data provided*

In [1]:
import pandas as pd

# Load the dataset
file_path = '/content/capstone_dataset.csv'  # replace with your CSV file path
df = pd.read_csv(file_path)

# Descriptive statistics
print("Descriptive Statistics:\n")
print(df.describe(include='all'))  # include='all' covers numeric & non-numeric columns

# Check for missing values
print("\nMissing Values Summary:\n")
missing = df.isnull().sum()
print(missing[missing > 0] if missing.any() else "No missing values found.")

Descriptive Statistics:

       Product type   SKU       Price  Availability  Number of products sold  \
count           100   100  100.000000    100.000000               100.000000   
unique            3   100         NaN           NaN                      NaN   
top         Chassis  SKU0         NaN           NaN                      NaN   
freq             40     1         NaN           NaN                      NaN   
mean            NaN   NaN   49.462461     48.400000               460.990000   
std             NaN   NaN   31.168193     30.743317               303.780074   
min             NaN   NaN    1.699976      1.000000                 8.000000   
25%             NaN   NaN   19.597823     22.750000               184.250000   
50%             NaN   NaN   51.239830     43.500000               392.500000   
75%             NaN   NaN   77.198228     75.000000               704.250000   
max             NaN   NaN   99.171329    100.000000               996.000000   

        Revenu

*2. Clean the dataset to remove duplicate rows or rows with missing data.*

In [2]:
# Check for duplicate rows
duplicates = df.duplicated()
num_duplicates = duplicates.sum()
print(f"\nNumber of duplicate rows: {num_duplicates}")

# Remove duplicate rows if any
if num_duplicates > 0:
    df = df.drop_duplicates()
    print("Duplicate rows removed.")
else:
    print("No duplicate rows found.")


Number of duplicate rows: 0
No duplicate rows found.


*3. Compute the average time to delivery by supplier (delivery time is the difference between when order date and expected delivery date)*

In [8]:
# Ensure the date columns are in datetime format
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Expected Delivery Date'] = pd.to_datetime(df['Expected Delivery Date'])

# Add new column: lead time to delivery (in days)
df['lead_time_to_del'] = (df['Expected Delivery Date'] - df['Order Date']).dt.days

# Preview
print(df[['Supplier name', 'Order Date', 'Expected Delivery Date', 'lead_time_to_del']].head())

# Save updated dataset to a new CSV
output_file = 'updated_dataset_with_lead_time.csv'
df.to_csv(output_file, index=False)

print(f"\nUpdated dataset saved to {output_file}")

  Supplier name Order Date Expected Delivery Date  lead_time_to_del
0    Supplier 3 2025-02-11             2025-02-23                12
1    Supplier 3 2025-06-14             2025-06-26                12
2    Supplier 1 2025-01-09             2025-01-18                 9
3    Supplier 5 2025-07-31             2025-08-15                15
4    Supplier 1 2025-05-01             2025-05-15                14

Updated dataset saved to updated_dataset_with_lead_time.csv


In [9]:
# Define function to calculate average lead time per supplier
def average_lead_time_per_supplier(dataframe):
    """
    Returns a DataFrame with average lead time to delivery per supplier.
    """
    return dataframe.pivot_table(
        index='Supplier name',
        values='lead_time_to_del',
        aggfunc='mean'
    ).reset_index()

# Call the function
avg_lead_times = average_lead_time_per_supplier(df)

# Display results
print("Average lead time per supplier:\n")
print(avg_lead_times)

Average lead time per supplier:

  Supplier name  lead_time_to_del
0    Supplier 1         10.518519
1    Supplier 2          9.272727
2    Supplier 3         10.600000
3    Supplier 4          9.444444
4    Supplier 5         10.888889


*4. Create an inventory management simulation. Randomly select demand from normal distribution (mean: 50, std: 10). Assume the holding cost is 10 dollars per unit per day. Assume the shortage cost is 50 dollars per unit per day.5. For each supplier, use their average days to delivery as the order lead time. For each supplier, run a 180-day simulation and calculate the inventory management costs for that supplier. [MONTE CARLO SIMULATION]*

In [10]:
import numpy as np

# -----------------------------
# Supplier lead times (from your table)
# -----------------------------
suppliers = pd.DataFrame({
    'Supplier name': ['Supplier 1', 'Supplier 2', 'Supplier 3', 'Supplier 4', 'Supplier 5'],
    'lead_time_to_del': [10.518519, 9.272727, 10.6, 9.444444, 10.888889]
})

# -----------------------------
# Simulation parameters
# -----------------------------
np.random.seed(42)  # reproducibility
days_to_simulate = 180
simulations = 50
mean_demand = 50
std_demand = 10
holding_cost_per_unit = 10
shortage_cost_per_unit = 50
initial_inventory = 100

# -----------------------------
# Monte Carlo Simulation Function
# -----------------------------
def run_inventory_simulation(lead_time, days=days_to_simulate, sims=simulations):
    """
    Run inventory simulation for a given supplier lead time.
    Returns average holding, shortage, and total costs across simulations.
    """
    results = []

    for sim in range(sims):
        inventory = initial_inventory
        total_holding_cost = 0
        total_shortage_cost = 0

        for day in range(1, days + 1):
            # Random daily demand
            demand = max(int(np.random.normal(mean_demand, std_demand)), 0)

            # Satisfy demand
            if inventory >= demand:
                inventory -= demand
                holding_cost = inventory * holding_cost_per_unit
                shortage_cost = 0
            else:
                shortage = demand - inventory
                inventory = 0
                holding_cost = 0
                shortage_cost = shortage * shortage_cost_per_unit

            total_holding_cost += holding_cost
            total_shortage_cost += shortage_cost

            # Replenishment every "lead_time" days (rounded)
            if day % int(round(lead_time)) == 0:
                inventory += int(mean_demand * lead_time)  # restock enough for avg demand during lead time

        total_cost = total_holding_cost + total_shortage_cost
        results.append(total_cost)

    return {
        'Average Cost': np.mean(results),
        'Min Cost': np.min(results),
        'Max Cost': np.max(results)
    }

# -----------------------------
# Run simulation for all suppliers
# -----------------------------
supplier_results = []

for _, row in suppliers.iterrows():
    supplier = row['Supplier name']
    lead_time = row['lead_time_to_del']

    sim_result = run_inventory_simulation(lead_time)
    sim_result['Supplier'] = supplier
    sim_result['Lead Time (days)'] = lead_time
    supplier_results.append(sim_result)

# Convert results to DataFrame
df_results = pd.DataFrame(supplier_results)

# -----------------------------
# Display results
# -----------------------------
print("\nMonte Carlo Simulation Results (180 days, 50 runs):\n")
print(df_results[['Supplier', 'Lead Time (days)', 'Average Cost', 'Min Cost', 'Max Cost']])


Monte Carlo Simulation Results (180 days, 50 runs):

     Supplier  Lead Time (days)  Average Cost  Min Cost  Max Cost
0  Supplier 1         10.518519      452384.8    428910    536620
1  Supplier 2          9.272727      671188.4    476950    879730
2  Supplier 3         10.600000      463061.2    436830    562560
3  Supplier 4          9.444444      847358.2    628820   1113660
4  Supplier 5         10.888889      520459.0    455890    676620


*6.   Determine which supplier has the lowest cost.*

In [11]:
# Find supplier with the lowest average cost
lowest_cost_supplier = df_results.loc[df_results['Average Cost'].idxmin()]

print("\nSupplier with the lowest inventory management cost:\n")
print(lowest_cost_supplier)


Supplier with the lowest inventory management cost:

Average Cost          452384.8
Min Cost                428910
Max Cost                536620
Supplier            Supplier 1
Lead Time (days)     10.518519
Name: 0, dtype: object
